In [ ]:
from PIL import Image, ImageOps
from transparent_background import Remover

remover = Remover()  # default setting
remover = Remover(mode='base') 

img = Image.open("../data/monin/combined_dataset/test/1725657501_6166165.png")
# img = img.resize((512, 512))
# mask = remover.process(img, type='map')
img

In [ ]:
import cv2

import numpy as np

low_threshold = 150
high_threshold = 400
# low_threshold = 550
# high_threshold = 650

# image = np.array(background_img)
image = np.array(img)
image = cv2.Canny(image, low_threshold, high_threshold)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
# Apply Gaussian blur to the Canny image (you can adjust kernel size (5, 5) and sigma as needed)
blurred_image = cv2.GaussianBlur(image, (5, 5), 0)

# Convert to PIL image for display or further processing
canny_image = Image.fromarray(blurred_image)

canny_image

In [ ]:


from diffusers import ControlNetModel
import os
import torch

from diffusers import StableDiffusionPipeline
from diffusers import DiffusionPipeline
from diffusers import StableDiffusionControlNetPipeline



controlnet = ControlNetModel.from_pretrained(
                                
                                            "lllyasviel/sd-controlnet-canny",
                                            # "diffusers/controlnet-canny-sdxl-1.0",
                                             torch_dtype=torch.float16
                                             )

pipe = StableDiffusionControlNetPipeline.from_pretrained("sd-model-finetuned", torch_dtype=torch.float16, controlnet=controlnet)
# pipe = DiffusionPipeline.from_pretrained(
#                                                 # "runwayml/stable-diffusion-v1-5", 
#                                                 "sd-model-finetuned",
#                                                     torch_dtype=torch.float16, 
#                                                 # controlnet=controlnet
# )


pipe = pipe.to("cuda")
# prompt = "isometric view of a skyscraper in the style of a city building game"
prompt = "Name: 'Luck Of The Irish' Frozen Lemonade; Recipie: 3/4 oz. Monin Pistachio Syrup\n3/4 oz. Monin Ginger Syrup\n2 oz. fresh lemon sour mix\n\nFill serving glass full of ice.\nPour ingredients into blender cup in order listed.\nAdd ice from serving glass, cap, and blend until smooth.\nPour back into serving glass, add garnish and serve.; Category: lemonade"
negative_prompt = 'low quality, bad quality, sketches'
controlnet_conditioning_scale = 0.5

images = pipe(prompt=prompt, negative_prompt=negative_prompt, image=canny_image, num_images_per_prompt=6).images